In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os

from models import *
from utils import progress_bar

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
lr = 0.1    # learning rate
resume = False # resume from checkpoint

In [4]:
# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='../data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=512, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='../data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [5]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

In [6]:
def test(epoch, net_name):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, '../checkpoint/ckpt-{}.pth'.format(net_name))
        best_acc = acc

In [60]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
def evaluation(model):
    model.eval()
    target_list = []
    prediction_list = []
    with torch.no_grad():
        for images, labels in testloader:
            model.eval()  # 进入测试模式
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1) 
            for i in labels:
                target_list.append(i.cpu().numpy())
            for i in predicted:
                prediction_list.append(i.cpu().numpy())


    rbf_svc_score = accuracy_score(target_list, prediction_list)

    print('Accuracy of the Residualnetwork on the 10000 test images: %.2f %%' % (rbf_svc_score * 100))

    # 输出分类报告
    print("Classification Report:")
    print(classification_report(target_list, prediction_list))

    # 输出混淆矩阵
    conf_matrix = confusion_matrix(target_list, prediction_list)
    print("Confusion Matrix:")
    print(conf_matrix)
    

In [6]:
# Model
print('==> Building model..')
net = ResNet50()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

if resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('../checkpoint/ckpt.pth')
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


for epoch in range(start_epoch, start_epoch+200):
    train(epoch)
    test(epoch, net_name='ResNet50')
    scheduler.step()

==> Building model..

Epoch: 0
 [================================================================>]  Step: 1s285ms | Tot: 22s437ms | Loss: 4.407 | Acc: 11.280% (5640/5000 98/98 
 [================================================================>]  Step: 34ms | Tot: 3s678ms | Loss: 2.289 | Acc: 13.270% (1327/1000 100/100 
Saving..

Epoch: 1
 [================================================================>]  Step: 195ms | Tot: 22s355ms | Loss: 2.250 | Acc: 15.966% (7983/5000 98/98 
 [================================================================>]  Step: 28ms | Tot: 3s610ms | Loss: 2.094 | Acc: 19.590% (1959/1000 100/100 
Saving..

Epoch: 2
 [================================================================>]  Step: 182ms | Tot: 21s31ms | Loss: 2.015 | Acc: 22.194% (11097/5000 98/98  
 [================================================================>]  Step: 32ms | Tot: 4s216ms | Loss: 1.919 | Acc: 24.940% (2494/1000 100/100 
Saving..

Epoch: 3
 [=====================================

In [62]:
net_resnet50 = ResNet50()
net_resnet50 = net_resnet50.to(device)
if device == 'cuda':
    net_resnet50 = torch.nn.DataParallel(net_resnet50)
    cudnn.benchmark = True
checkpoint_resnet50 = torch.load('../checkpoint/ckpt-ResNet50.pth')
net_resnet50.load_state_dict(checkpoint_resnet50['net'], strict=False)

evaluation(net_resnet50)

Accuracy of the Residualnetwork on the 10000 test images: 95.00 %
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.95      0.95      1000
           1       0.98      0.98      0.98      1000
           2       0.95      0.93      0.94      1000
           3       0.91      0.88      0.89      1000
           4       0.94      0.96      0.95      1000
           5       0.91      0.91      0.91      1000
           6       0.96      0.98      0.97      1000
           7       0.97      0.96      0.97      1000
           8       0.96      0.97      0.97      1000
           9       0.97      0.97      0.97      1000

    accuracy                           0.95     10000
   macro avg       0.95      0.95      0.95     10000
weighted avg       0.95      0.95      0.95     10000

Confusion Matrix:
[[951   0  10   5   2   0   0   2  23   7]
 [  3 978   0   0   0   0   0   0   2  17]
 [ 10   0 933   9  15  12  12   6   3   0]
 [  4

In [7]:
# Model
print('==> Building model..')
net = ResNet34()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

if resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('../checkpoint/ckpt.pth')
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


for epoch in range(start_epoch, start_epoch+200):
    train(epoch)
    test(epoch, net_name='ResNet34')
    scheduler.step()


==> Building model..

Epoch: 0
 [================================================================>]  Step: 377ms | Tot: 11s762ms | Loss: 2.493 | Acc: 16.944% (8472/5000 98/98 
 [================================================================>]  Step: 24ms | Tot: 2s557ms | Loss: 1.922 | Acc: 27.140% (2714/1000 100/100 

Epoch: 1
 [================================================================>]  Step: 99ms | Tot: 11s773ms | Loss: 1.783 | Acc: 32.790% (16395/5000 98/98  
 [================================================================>]  Step: 19ms | Tot: 2s482ms | Loss: 1.640 | Acc: 38.110% (3811/1000 100/100 

Epoch: 2
 [================================================================>]  Step: 110ms | Tot: 11s406ms | Loss: 1.604 | Acc: 39.956% (19978/5000 98/98 
 [================================================================>]  Step: 29ms | Tot: 2s605ms | Loss: 1.481 | Acc: 44.140% (4414/1000 100/100 

Epoch: 3
 [================================================================>

In [63]:
net_resnet34 = ResNet34()
net_resnet34 = net_resnet34.to(device)
if device == 'cuda':
    net_resnet34 = torch.nn.DataParallel(net_resnet34)
    cudnn.benchmark = True
checkpoint_resnet34 = torch.load('../checkpoint/ckpt-ResNet34.pth')
net_resnet34.load_state_dict(checkpoint_resnet34['net'], strict=False)

evaluation(net_resnet34)

Accuracy of the Residualnetwork on the 10000 test images: 95.18 %
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.96      0.96      1000
           1       0.98      0.98      0.98      1000
           2       0.94      0.94      0.94      1000
           3       0.91      0.89      0.90      1000
           4       0.96      0.97      0.96      1000
           5       0.91      0.92      0.91      1000
           6       0.96      0.97      0.96      1000
           7       0.97      0.96      0.97      1000
           8       0.96      0.97      0.97      1000
           9       0.98      0.96      0.97      1000

    accuracy                           0.95     10000
   macro avg       0.95      0.95      0.95     10000
weighted avg       0.95      0.95      0.95     10000

Confusion Matrix:
[[960   0  11   5   1   0   1   2  17   3]
 [  2 982   0   0   1   0   0   0   4  11]
 [ 11   0 939  10  12  14   9   3   2   0]
 [  6

In [8]:
# Model
print('==> Building model..')
net = GoogLeNet()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

if resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('../checkpoint/ckpt.pth')
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


for epoch in range(start_epoch, start_epoch+200):
    train(epoch)
    test(epoch, net_name='GoogLeNet')
    scheduler.step()

==> Building model..

Epoch: 0
 [================================================================>]  Step: 1s775ms | Tot: 18s471ms | Loss: 1.478 | Acc: 45.434% (22717/5000 98/98 
 [================================================================>]  Step: 38ms | Tot: 3s797ms | Loss: 1.360 | Acc: 52.930% (5293/1000 100/100 
Saving..

Epoch: 1
 [================================================================>]  Step: 124ms | Tot: 16s885ms | Loss: 0.911 | Acc: 67.554% (33777/5000 98/98 
 [================================================================>]  Step: 34ms | Tot: 3s805ms | Loss: 1.229 | Acc: 60.720% (6072/1000 100/100 
Saving..

Epoch: 2
 [================================================================>]  Step: 169ms | Tot: 16s795ms | Loss: 0.689 | Acc: 75.902% (37951/5000 98/98 
 [================================================================>]  Step: 44ms | Tot: 3s756ms | Loss: 0.780 | Acc: 74.300% (7430/1000 100/100 
Saving..

Epoch: 3
 [===================================

In [65]:
net_GoogLeNet = GoogLeNet()
net_GoogLeNet = net_GoogLeNet.to(device)
if device == 'cuda':
    net_GoogLeNet = torch.nn.DataParallel(net_GoogLeNet)
    cudnn.benchmark = True
checkpoint_GoogLeNet = torch.load('../checkpoint/ckpt-GoogLeNet.pth')
net_GoogLeNet.load_state_dict(checkpoint_GoogLeNet['net'], strict=False)

evaluation(net_GoogLeNet)

Accuracy of the Residualnetwork on the 10000 test images: 95.27 %
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      1000
           1       0.97      0.97      0.97      1000
           2       0.96      0.94      0.95      1000
           3       0.90      0.89      0.89      1000
           4       0.95      0.96      0.96      1000
           5       0.91      0.93      0.92      1000
           6       0.96      0.97      0.97      1000
           7       0.98      0.96      0.97      1000
           8       0.98      0.97      0.97      1000
           9       0.97      0.97      0.97      1000

    accuracy                           0.95     10000
   macro avg       0.95      0.95      0.95     10000
weighted avg       0.95      0.95      0.95     10000

Confusion Matrix:
[[975   3   5   3   0   0   1   0  10   3]
 [  4 972   0   0   0   0   0   0   5  19]
 [ 11   0 936  16  12   8  14   2   1   0]
 [  3